<a href="https://www.kaggle.com/code/soumyadeepsarkar12/infer-umiam?scriptVersionId=162365517" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hehehe/cleandata.csv
/kaggle/input/kriti-24/sample_submission.csv
/kaggle/input/kriti-24/train.csv
/kaggle/input/kriti-24/test.csv
/kaggle/input/hello1/model1.pth
/kaggle/input/hello1/model2.pth
/kaggle/input/hello1/model4.pth
/kaggle/input/hello1/model5.pth
/kaggle/input/hello1/state.db
/kaggle/input/hello1/submission.csv
/kaggle/input/hello1/submission_ordered_new.csv


In [2]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import os
import torch
import re
import string
import json

import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import torch.nn as nn
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


In [3]:
df=pd.read_csv('/kaggle/input/hehehe/cleandata.csv')

In [4]:
df

,Text,q-bio.GN,stat.AP,q-fin.TR,math.GR,q-bio.NC,math.NT,q-fin.MF,cs.SE,math.LO,...,cs.CL,econ.GN,q-fin.GN,math.ST,econ.TH,math.QA,math.IT,stat.TH,math.CO,q-fin.RM
0,axiomatic aspects default inference paper stud...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,extensions group infinite conjugacy classes ch...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,complexvalued cnns rf datadriven wireless devi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,reconstruction drift diffusion transition prob...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,three classes propagation rules grs egrs codes...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,generalized fourier integral operators spaces ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51206,weaklysupervised 3d visual grounding visual li...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
51207,strongly pseudoconvex handlebodies explicit co...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51208,improving endtoend speech processing efficient...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51210 entries, 0 to 51209
Data columns (total 58 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      51210 non-null  object
 1   q-bio.GN  51210 non-null  int64 
 2   stat.AP   51210 non-null  int64 
 3   q-fin.TR  51210 non-null  int64 
 4   math.GR   51210 non-null  int64 
 5   q-bio.NC  51210 non-null  int64 
 6   math.NT   51210 non-null  int64 
 7   q-fin.MF  51210 non-null  int64 
 8   cs.SE     51210 non-null  int64 
 9   math.LO   51210 non-null  int64 
 10  econ.EM   51210 non-null  int64 
 11  q-fin.PM  51210 non-null  int64 
 12  cs.CE     51210 non-null  int64 
 13  q-bio.MN  51210 non-null  int64 
 14  cs.CV     51210 non-null  int64 
 15  math.PR   51210 non-null  int64 
 16  eess.SP   51210 non-null  int64 
 17  math.AT   51210 non-null  int64 
 18  cs.SD     51210 non-null  int64 
 19  stat.CO   51210 non-null  int64 
 20  q-fin.EC  51210 non-null  int64 
 21  cs.OS     51

In [ ]:
import pandas as pd

# Specify the list of columns you want to check for 1s
columns_to_check = ['q-bio.GN', 'q-fin.TR', 'q-bio.NC', 'q-fin.MF', 'q-fin.PM', 'cs.CE', 'q-bio.MN', 'math.AT',
                    'stat.CO', 'q-fin.EC', 'cs.OS', 'q-fin.CP', 'q-bio.CB', 'q-bio.BM', 'cs.DM', 'cs.GT', 'cs.AR',
                    'q-bio.TO', 'q-fin.PR', 'econ.GN', 'q-fin.GN', 'econ.TH', 'q-fin.RM']

# Create a mask to identify rows where at least one of the specified columns has a value of 1
mask = df[columns_to_check].any(axis=1)

# Filter the DataFrame to keep only the rows where the mask is True
filtered_df = df[mask]

# If you want to drop the original columns that were checked, you can use:
# filtered_df = filtered_df.drop(columns=columns_to_check)

# Now, filtered_df contains only the rows where at least one of the specified columns has a value of 1
print(filtered_df)


In [ ]:
filtered_df.reset_index(drop=True, inplace=True)

# The 'drop=True' argument will drop the old index, and 'inplace=True' will modify the DataFrame in place.

# Now, the filtered_df has a reset index starting from 0.
print(filtered_df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Calculate the frequency of each class
class_frequencies = filtered_df.iloc[:, 1:-1].sum().sort_values(ascending=False)

# Plot class frequencies
plt.figure(figsize=(12, 8))
class_frequencies.plot(kind='bar')
plt.title('Frequency of Each Class')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

class_frequencies

In [ ]:
# Assuming `df` is your DataFrame and `columns_to_check` is your list of columns
# Create a mask to identify rows where all specified columns do not have a value of 1
mask = df[columns_to_check].sum(axis=1) == 0

# Use the mask to filter the DataFrame
df_majority = df[mask]

# Now `df_filtered` contains rows where none of the specified columns have a value of 1


In [ ]:
df_majority.reset_index(drop=True, inplace=True)

In [ ]:
df_majority

In [ ]:
# Calculate the frequency of each class
class_frequencies = df_majority.iloc[:, 1:-1].sum().sort_values(ascending=False)

# Plot class frequencies
plt.figure(figsize=(12, 8))
class_frequencies.plot(kind='bar')
plt.title('Frequency of Each Class')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

class_frequencies

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
target_cols = [col for col in df.columns if col not in ['Text']]
len(target_cols)

In [ ]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 3e-9
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
# class BERTDataset(Dataset):
#     def __init__(self, df, tokenizer, max_len):
#         self.df = df
#         self.max_len = max_len
#         self.text = df.Text
#         self.tokenizer = tokenizer
#         self.targets = df[target_cols].values
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, index):
#         text = self.text[index]
#         inputs = self.tokenizer.encode_plus(
#             text,
#             truncation=True,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_token_type_ids=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         token_type_ids = inputs["token_type_ids"]
        
#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
#             'targets': torch.tensor(self.targets[index], dtype=torch.float)
#         }

In [ ]:
# train_dataset_minor = BERTDataset(df, tokenizer, MAX_LEN)


In [ ]:
# majority = DataLoader(train_dataset_minor, batch_size=TRAIN_BATCH_SIZE, 
#                           num_workers=4, shuffle=True, pin_memory=True)

In [ ]:
# import torch
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import train_test_split
# from transformers import AutoModel


In [ ]:
# class MultiLabelSciBERT(torch.nn.Module):
#     def __init__(self, num_labels):
#         super(MultiLabelSciBERT, self).__init__()
#         self.sci_bert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
#         self.classifier = torch.nn.Linear(768, num_labels)

#     def forward(self, ids,mask):
#         outputs = self.sci_bert(ids, attention_mask=mask)
#         sequence_output = outputs[0][:, 0, :]  
#         return self.classifier(sequence_output)
    
    
    


# FROM HERE FOR INFERENCE

In [ ]:
model_news= MultiLabelSciBERT(57)
model_path = '/kaggle/input/hello1/model5.pth'
loaded_st = torch.load(model_path)
model_news.load_state_dict(loaded_st)
model_news.to(device)

In [ ]:
# def validation():
#     model_news.eval() 
#     fin_targets=[]
#     fin_outputs=[]
#     with torch.no_grad():
#         for _, data in enumerate(majority, 0):
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)
#             outputs = model_news(ids, mask)
# #              outputs = model(ids, mask)
#             fin_targets.extend(targets.cpu().detach().numpy().tolist())
#             fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
#     return fin_outputs, fin_targets

In [ ]:
# import torch
# from torch.cuda.amp import autocast
# import torch
# from torch.cuda.amp import GradScaler

# # Define the GradScaler object
# scaler = GradScaler()
# def train(epoch):
#     model_news.train()
#     for _, data in enumerate(majority, 0):
#         ids = data['ids'].to(device, dtype=torch.long)
#         mask = data['mask'].to(device, dtype=torch.long)
#         targets = data['targets'].to(device, dtype=torch.float)

#         optimizer.zero_grad()

#         with autocast():
#             outputs = model_news(ids, mask)
#             loss = loss_fn(outputs, targets )

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         if _ % 500 == 0:
#             print(f'Epoch: {epoch}, Loss: {loss.item()}')


In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model_news.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
# for epoch in range(EPOCHS):
#     train(epoch) 

In [ ]:
# def validation():
#     model_news.eval() 
#     fin_targets=[]
#     fin_outputs=[]
#     with torch.no_grad():
#         for _, data in enumerate(majority, 0):
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)
#             outputs = model_news(ids, mask)
# #              outputs = model(ids, mask)
#             fin_targets.extend(targets.cpu().detach().numpy().tolist())
#             fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
#     return fin_outputs, fin_targets

In [ ]:
# outputs, targets = validation()
# outputs = np.array(outputs) >= 0.5
# accuracy = metrics.accuracy_score(targets, outputs)
# f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
# f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
# print(f"Accuracy Score = {accuracy}")
# print(f"F1 Score (Micro) = {f1_score_micro}")
# print(f"F1 Score (Macro) = {f1_score_macro}")

# TRAINING ON MINORITY CLASSES 

In [ ]:
# import torch
# from torch.cuda.amp import autocast
# import torch
# from torch.cuda.amp import GradScaler

# # Define the GradScaler object
# scaler = GradScaler()
# def train(epoch):
#     model.train()
#     for _, data in enumerate(train_loader, 0):
#         ids = data['ids'].to(device, dtype=torch.long)
#         mask = data['mask'].to(device, dtype=torch.long)
#         targets = data['targets'].to(device, dtype=torch.float)

#         optimizer.zero_grad()

#         with autocast():
#             outputs = model(ids, mask)
#             loss = loss_fn(outputs, targets )

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         if _ % 500 == 0:
#             print(f'Epoch: {epoch}, Loss: {loss.item()}')


# RUNNING ON TEST SET

In [ ]:
def predict():
    model_news.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(majority, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model_news(ids, mask)
            predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return predictions

# Get predictions
test_predictions = predict()


In [ ]:
train_csv=pd.read_csv('/kaggle/input/kriti-24/train.csv')

In [ ]:
train_csv

In [ ]:
import pandas as pd

# Assuming you have two DataFrames: df and train_csv
# Add the 'Id' column from train_csv to df
df['Id'] = train_csv['Id']


In [ ]:
df

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols)
submission_df.insert(0, 'Text', df.Text)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
target_cols2 = [col for col in df.columns if col not in ['Text','Title','Abstract','Id']]
len(target_cols2)

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols2)
submission_df.insert(0, 'Id', df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
template_df = pd.read_csv('/kaggle/input/kriti-24/sample_submission.csv')
required_column_order = template_df.columns.tolist()


In [ ]:
# # Ensure that 'Id' is the first column
# required_column_order.remove('Id')
# required_column_order.insert(0, 'Id')

# Reorder your DataFrame columns
train_predict = submission_df[required_column_order]


In [ ]:
train_predict.loc[:, 'Text'] = df['Text'].copy()


In [ ]:
train_predict.drop(['Id'],axis=1)

In [ ]:
last_column = train_predict.pop(train_predict.columns[-1])
train_predict.insert(0, last_column.name, last_column)

In [ ]:
train_predict
save

In [ ]:
df

# DOING A PREDICTION 

In [ ]:
import pandas as pd
# Load the test dataset
test_df = pd.read_csv("/kaggle/input/kriti-24/test.csv")

# Preprocess the text data
test_df['Text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df = test_df.drop(['Title', 'Abstract'], axis=1)
test_df['Text']=test_df['Text'].str.lower()


In [ ]:
def remove_punct(text):
    punctuation =string.punctuation
    return text.translate(str.maketrans('' , '',punctuation))
test_df['Text']=test_df['Text'].apply(lambda x: remove_punct(x))

In [ ]:
from nltk.corpus import stopwords 
",".join(stopwords.words('english'))
STOPWRDS=set(stopwords.words('english'))

In [ ]:
def remove_stp(Text):
    return " ".join([word for word in Text.split() if word not in STOPWRDS])

In [ ]:
test_df['Text']=test_df['Text'].apply(lambda x: remove_stp(x))

In [ ]:
not_needed = [
     "using", "results", "performance", "show", "method", "approach",
    "also", "based", "problem", "proposed", "two", "information", "new", "large",
    "different", "study", "however",  "analysis", "one", "work", "used", "first",
    "set", "use", "existing", "present", "process", "demonstrate", "task", "general",
    "several", "due", "compared", "via", "moreover", "eg", "thus", "possible", "make",
    "like", "important", "key", "furthermore", "give", "state", "certain", "ie", "form",
    "allows", "finally", "often", "even", "many", "various", "well", "also", "however",
    "several", "due", "across", "may", "without", "among", "including", "particular",
    "especially", "either", "often", "even", "moreover", "thus", "ie", "eg", "although",
    "despite","would" ,"within" ,"",
]

import pandas as pd
def not_need(Text):
    return " ".join([word for word in Text.split() if word not in not_needed])

test_df['Text'] = test_df['Text'].apply(lambda x:not_need(x))

In [ ]:
class BERTTestDatasets(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

# Create the test dataset
result_dataset = BERTTestDatasets(test_df, tokenizer, MAX_LEN)


In [ ]:
result_loader = DataLoader(result_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=4)


In [ ]:
def predict():
    model_news.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(result_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model_news(ids, mask)
            predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return predictions

# Get predictions
test_predictions = predict()


In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
target_cols2 = [col for col in df.columns if col not in ['Text','Title','Abstract']]
len(target_cols2)

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols2)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
template_df = pd.read_csv('/kaggle/input/kriti-24/sample_submission.csv')
required_column_order = template_df.columns.tolist()


In [ ]:
# Ensure that 'Id' is the first column
required_column_order.remove('Id')
required_column_order.insert(0, 'Id')

# Reorder your DataFrame columns
submission_df = submission_df[required_column_order]


# SUBMISSION_ORDER_NEW is the main submission file

In [ ]:
submission_df
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
